# Create walking times:

This notebook computes the time it takes to walk between stations that are under 500m from each-other. 

In [1]:
%%configure
{"conf": {
    "spark.app.name": "dslab-group_final"
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8919,application_1589299642358_3451,pyspark,idle,Link,Link,
8973,application_1589299642358_3509,pyspark,idle,Link,Link,
8983,application_1589299642358_3520,pyspark,idle,Link,Link,
8996,application_1589299642358_3534,pyspark,idle,Link,Link,
9002,application_1589299642358_3540,pyspark,idle,Link,Link,
9003,application_1589299642358_3541,pyspark,idle,Link,Link,
9028,application_1589299642358_3573,pyspark,idle,Link,Link,
9035,application_1589299642358_3582,pyspark,busy,Link,Link,
9036,application_1589299642358_3583,pyspark,idle,Link,Link,
9045,application_1589299642358_3590,pyspark,idle,Link,Link,


In [2]:
stops = spark.read.format('orc').load('/data/sbb/timetables/orc/stops/000000_0')
transfers = spark.read.format('orc').load('/data/sbb/timetables/orc/transfers/000000_0')

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9076,application_1589299642358_3630,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Imports:

In [3]:
from geopy.distance import distance as geo_distance
from pyspark.sql import Row
from pyspark.sql.functions import col
from pyspark.sql.functions import udf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Calculate the distance between all stops in order to select those under 500m distance (as the crow flies).

In [4]:
def zurich_distance(x, y):
    """zurich_distance: returns the distance of a station to Zurich HB
    @input: (lat,lon) of a station
    @output: distance in km to Zurich HB
    """
    zurich_pos = (47.3781762039461, 8.54019357578468)
    return geo_distance(zurich_pos, (x,y)).km

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
@udf("float")
def compute_distance(x1, y1, x2, y2):
    """
    Compute distance takes as input two pairs of latitude and longitude coming
    from two different stops and compute the distance between those two stops in meters 
    """
    return geo_distance((x1, y1), (x2,y2)).m

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use latitude and longitude to filter stops that are only within 15 km of Zurich HB

In [6]:
stops_distance = stops.rdd.map(lambda x: (x['stop_id'], zurich_distance(x['stop_lat'], x['stop_lon'])))
stops_distance = spark.createDataFrame(stops_distance.map(lambda r: Row(stop_id=r[0], 
                                                                        zurich_distance=r[1])))
stops_distance = stops_distance.filter(col('zurich_distance') <= 15)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Create DataFrame containing all possible pairs of stops with their latitude and longitude and then compute the distance between old pairs of stations

In [7]:
stops_pos = stops.join(stops_distance, 'stop_id').select(col('stop_id'), 
                                                         col('stop_lat'), col('stop_lon'))
stops_pos = stops_pos.select(col('stop_id').alias('stop_id_1'), 
                             col('stop_lat').alias('stop_lat_1'), 
                             col('stop_lon').alias('stop_lon_1'))
stops_pos = stops_pos.crossJoin(stops_pos.select(col('stop_id_1').alias('stop_id_2'), 
                                                 col('stop_lat_1').alias('stop_lat_2'), col('stop_lon_1').alias('stop_lon_2')))
stops_pos = stops_pos.where(col('stop_id_1') != col('stop_id_2'))
stops_pos_dist = stops_pos.withColumn('distance', 
                                      compute_distance(col('stop_lat_1'), 
                                                       col('stop_lon_1'), 
                                                       col('stop_lat_2'), 
                                                       col('stop_lon_2')))
stops_pos_dist.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[stop_id_1: string, stop_lat_1: double, stop_lon_1: double, stop_id_2: string, stop_lat_2: double, stop_lon_2: double, distance: float]

Keep only pairs for which the distance is less than 500 meters and compute the duration using a speed of 50 meters per minutes

In [8]:
walking_edges = stops_pos_dist.select(col('stop_id_1').alias('source'), col('stop_id_2').alias('target'), 
                                      col('distance'))\
                                        .where(col('distance') <= 500)\
                                        .withColumn('duration', col('distance')/50)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
transfers = transfers.select('from_stop_id', 'to_stop_id', 'min_transfer_time')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
%%spark -o transfers -n 30000

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
walking_edges.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12534

In [15]:
%%spark -o walking_edges -n 30000

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

KeyboardInterrupt: 

If there is available information for a transfer between stations, we add it to `walking_edges`. 

In [14]:
%%local
merged = transfers.merge(walking_edges, 
                         left_on=['from_stop_id', 'to_stop_id'], 
                         right_on=['source', 'target'], how='right')

mask_transfers = merged.to_stop_id.notnull() & merged.from_stop_id.notnull()

merged.loc[mask_transfers, 'walk_duration'] = merged.loc[mask_transfers, 'min_transfer_time'] / 60
merged.loc[~mask_transfers, 'walk_duration'] = merged.loc[~mask_transfers, 'duration'] + 2

NameError: name 'walking_edges' is not defined

## Save to local:

In [80]:
%%spark -o walking_edges -n 1000000

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
%%local
walking_edges.to_pickle('walking_times.pickle')